<a href="https://colab.research.google.com/github/hz6450/2023-1-ImmersiveCamp/blob/main/%EC%96%BC%EA%B5%B4%EC%9D%B8%EC%8B%9D%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EB%94%A5%ED%8E%98%EC%9D%B4%ED%81%AC_%EB%B0%8F_%ED%95%84%ED%84%B0%EC%A0%81%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/yoyo-nb/Thin-Plate-Spline-Motion-Model.git

fatal: destination path 'Thin-Plate-Spline-Motion-Model' already exists and is not an empty directory.


In [ ]:
cd Thin-Plate-Spline-Motion-Model

/content/Thin-Plate-Spline-Motion-Model


In [ ]:
!mkdir checkpoints
!pip3 install wldhx.yadisk-direct
!curl -L $(yadisk-direct https://disk.yandex.com/d/O8K-gAIXBIKNxg) -o checkpoints/vox.pth.tar
# !curl -L $(yadisk-direct https://disk.yandex.com/d/vk5dirE6KNvEXQ) -o checkpoints/taichi.pth.tar
# !curl -L $(yadisk-direct https://disk.yandex.com/d/IVtro0k2MVHSvQ) -o checkpoints/mgif.pth.tar
# !curl -L $(yadisk-direct https://disk.yandex.com/d/B3ipFzpmkB1HIA) -o checkpoints/ted.pth.tar

mkdir: cannot create directory ‘checkpoints’: File exists
Traceback (most recent call last):
  File "/usr/local/bin/yadisk-direct", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/wldhx/yadisk_direct/main.py", line 23, in main
    print(*[get_real_direct_link(x) for x in args.sharing_link], sep=args.separator)
  File "/usr/local/lib/python3.10/dist-packages/wldhx/yadisk_direct/main.py", line 23, in <listcomp>
    print(*[get_real_direct_link(x) for x in args.sharing_link], sep=args.separator)
  File "/usr/local/lib/python3.10/dist-packages/wldhx/yadisk_direct/main.py", line 12, in get_real_direct_link
    return pk_request.json()['href']
KeyError: 'href'
curl: no URL specified!
curl: try 'curl --help' or 'curl --manual' for more information


In [ ]:
import torch

# edit the config
device = torch.device('cuda:0')
dataset_name = 'vox' # ['vox', 'taichi', 'ted', 'mgif']
source_image_path = './assets/source.png'
driving_video_path = './assets/driving.mp4'
output_video_path = './generated.mp4'
config_path = 'config/vox-256.yaml'
checkpoint_path = 'checkpoints/vox.pth.tar'
predict_mode = 'relative' # ['standard', 'relative', 'avd']
find_best_frame = False # when use the relative mode to animate a face, use 'find_best_frame=True' can get better quality result

pixel = 256 # for vox, taichi and mgif, the resolution is 256*256
if(dataset_name == 'ted'): # for ted, the resolution is 384*384
    pixel = 384

if find_best_frame:
  !pip install face_alignment

In [ ]:
try:
  import imageio
  import imageio_ffmpeg
except:
  !pip install imageio_ffmpeg
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
import os

warnings.filterwarnings("ignore")

source_image = imageio.imread(source_image_path)
reader = imageio.get_reader(driving_video_path)

source_image = resize(source_image, (pixel, pixel))[..., :3]

fps = reader.get_meta_data()['fps']
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()

driving_video = [resize(frame, (pixel, pixel))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani


HTML(display(source_image, driving_video).to_html5_video())

In [ ]:
from demo import load_checkpoints
inpainting, kp_detector, dense_motion_network, avd_network = load_checkpoints(config_path = config_path, checkpoint_path = checkpoint_path, device = device)

In [ ]:
from demo import make_animation
from skimage import img_as_ubyte

if predict_mode=='relative' and find_best_frame:
    from demo import find_best_frame as _find
    i = _find(source_image, driving_video, device.type=='cpu')
    print ("Best frame: " + str(i))
    driving_forward = driving_video[i:]
    driving_backward = driving_video[:(i+1)][::-1]
    predictions_forward = make_animation(source_image, driving_forward, inpainting, kp_detector, dense_motion_network, avd_network, device = device, mode = predict_mode)
    predictions_backward = make_animation(source_image, driving_backward, inpainting, kp_detector, dense_motion_network, avd_network, device = device, mode = predict_mode)
    predictions = predictions_backward[::-1] + predictions_forward[1:]
else:
    predictions = make_animation(source_image, driving_video, inpainting, kp_detector, dense_motion_network, avd_network, device = device, mode = predict_mode)

#save resulting video
# Save resulting video
imageio.mimsave('test.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)


HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 169/169 [00:11<00:00, 14.44it/s]


In [ ]:
import cv2
import numpy as np
from IPython.display import HTML, display
from base64 import b64encode

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def apply_mosaic(image, x, y, width, height, scale):
    roi = image[y:y+height, x:x+width]
    small_roi = cv2.resize(roi, (scale, scale), interpolation=cv2.INTER_LINEAR)
    mosaic_roi = cv2.resize(small_roi, (width, height), interpolation=cv2.INTER_NEAREST)
    image[y:y+height, x:x+width] = mosaic_roi
    return image

# 비디오 파일 열기
cap = cv2.VideoCapture('test.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)

# 결과물을 저장할 비디오 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('mosaic_result.mp4', fourcc, fps, (int(cap.get(3)), int(cap.get(4))))

# 비디오 프레임 반복 처리
while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    # 프레임을 BGR 형식으로 변환
    frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)

    # 얼굴 검출을 위해 이미지를 8-bit 그레이스케일로 변환
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # 검출된 얼굴들에 대해 모자이크 처리 수행
    for (x, y, w, h) in faces:
        frame_bgr = apply_mosaic(frame_bgr, x, y, w, h, scale=10)

    # 모자이크 처리된 프레임을 결과 비디오에 쓰기
    out.write(frame_bgr)

# 비디오 파일 닫기
cap.release()
out.release()



In [ ]:
import cv2
import numpy as np
from IPython.display import HTML, display
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import imageio

# Load the mosaic result video
mosaic_result_path = 'mosaic_result.mp4'
mosaic_result_reader = imageio.get_reader(mosaic_result_path)
mosaic_result_frames = [frame for frame in mosaic_result_reader]

# Display the mosaic result video
def display_mosaic_result(frames):
    fig = plt.figure(figsize=(8, 6))
    ims = []
    for frame in frames:
        im = plt.imshow(frame, animated=True)
        plt.axis('off')
        ims.append([im])
    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

HTML(display_mosaic_result(mosaic_result_frames).to_html5_video())
